In [1]:
import pandas as pd
import sys

sys.path.append("/Users/sva21/Documents/mangrove/code/backtester-RL/")

## Orders


In [2]:
import src.order

In [3]:
order_type = "bid"

qty_b1 = 100
price_b1 = 99
bid1 = src.order.Order(order_type, qty_b1, price_b1)

qty_b2 = 10
price_b2 = 98
bid2 = src.order.Order(order_type, qty_b2, price_b2)

order_type = "ask"

qty_a1 = 100
price_a1 = 101
ask1 = src.order.Order(order_type, qty_a1, price_a1)

qty_a2 = 10
price_a2 = 102
ask2 = src.order.Order(order_type, qty_a2, price_a2)

## Order Book


In [4]:
from src.order_book import OrderBook

orderbook = OrderBook([bid1, bid2], [ask1, ask2])
print(orderbook)

┌─────────┬────────┬─────────┬─────────┐
│   Index │   Size │   Price │   Total │
├─────────┼────────┼─────────┼─────────┤
│       0 │     10 │     102 │    1020 │
│       1 │    100 │     101 │   10100 │
│       2 │    100 │      99 │    9900 │
│       3 │     10 │      98 │     980 │
└─────────┴────────┴─────────┴─────────┘


## Main Script


In [5]:
from src.kandel import kandel_simulator
from src.time_series import load_csv

In [6]:
ts = load_csv("data/ETHUSDC-1s-2024-05>08-clean.csv")

In [7]:
ts

n_rows: 10627200 : n_cols = 1 : unit : (1, 's') 
                        price
time                        
2024-05-01 00:00:00  3011.83
2024-05-01 00:00:01  3011.40
2024-05-01 00:00:02  3011.00
2024-05-01 00:00:03  3011.00
2024-05-01 00:00:04  3011.40
...                      ...
2024-08-31 23:59:55  2513.13
2024-08-31 23:59:56  2513.13
2024-08-31 23:59:57  2513.13
2024-08-31 23:59:58  2513.13
2024-08-31 23:59:59  2513.13

[10627200 rows x 1 columns]

In [8]:
historical_vol = ts.to_pandas()["price"].rolling(3600 * 24).std()
implied_vol = pd.read_csv(
    "data/ETHBVOLUSDT-BVOLIndex-2024-08-1>31.csv", index_col=0, parse_dates=True
)

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_72656/3024349770.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  implied_vol = pd.read_csv(


In [9]:
window = 3600 * 24
quote = 37500  # USDC
base = 37500  # also in USDC, will be swapped to base at day 0
vol_mult = 1.645
n_points = 10
step_size = 1
days = 60

transactions, res, order_book_history = kandel_simulator(
    ts=ts[-(days + 1) * 24 * 3600 :],
    quote=quote,
    base=base,
    vol_mult=vol_mult,
    n_points=n_points,
    step_size=step_size,
    window=window,
)
res.to_pandas().to_csv("results/simul_results.csv")

100%|██████████| 5184000/5184000 [01:22<00:00, 62817.73it/s]


# Results


In [10]:
import pandas as pd  # type: ignore

pd.options.plotting.backend = "plotly"

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [11]:
res = pd.read_csv("results/simul_results.csv", index_col=0, parse_dates=True)

In [12]:
res["mtm_eth"] = res["mtm"] / res["price"]
res["base_returns"] = res["mtm_eth"] / ((quote + base) / res["price"][window]) - 1
res["quote_returns"] = res["mtm"] / (quote + base) - 1

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_72656/4188196617.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  res["base_returns"] = res["mtm_eth"] / ((quote + base) / res["price"][window]) - 1


In [13]:
window_hours = window // 3600 - 1
res_1h = res.resample("1h").last()

# PLOT
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{}], [{}]],
    vertical_spacing=0.05,
    subplot_titles=("MTM", "Returns", "Price"),
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm[window_hours:],
        mode="lines",
        name="MTM in USDC",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm_eth[window_hours:],
        mode="lines",
        name="MTM in ETH",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.quote_returns[window_hours:],
        mode="lines",
        name="Over holding USDC",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.base_returns[window_hours:],
        mode="lines",
        name="Over holding ETH",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
    ),
    row=3,
    col=1,
)

fig.update_layout(
    height=1000,
    width=1400,
    title_text=f"MTM, Returns and Price.<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.update_yaxes(row=2, col=1, tickformat=".2%")

fig.show()

fig.write_image(
    f"results/saves/{window}_{quote}_{base}_{vol_mult}_{n_points}_{step_size}_{days}.png",
    width=1400,
    height=1000,
)

In [14]:
order_book_history_df = pd.DataFrame.from_records(
    [ob.to_dict() for ob in order_book_history]
)
order_book_history_df.index = res[window - 1 :].index
order_book_history_1h = order_book_history_df.resample("1h").last()

In [15]:
order_book_history_1h

,bids,asks
time,,
2024-07-02 23:00:00,"[3408.903154, 3401.482497, 3394.077994, 3386.6...","[3423.79307, 3431.2624, 3438.748024, 3446.2499..."
2024-07-03 00:00:00,"[3408.903154, 3401.482497, 3394.077994, 3386.6...","[3431.2624, 3438.748024, 3446.24998, 3453.7683..."
2024-07-03 01:00:00,"[3371.961054, 3364.620814, 3357.296553, 3349.9...","[3394.077994, 3401.482497, 3408.903154, 3416.3..."
2024-07-03 02:00:00,"[3357.296553, 3349.988236, 3342.695828]","[3379.317307, 3386.689609, 3394.077994, 3401.4..."
2024-07-03 03:00:00,[3342.695828],"[3364.620814, 3371.961054, 3379.317307, 3386.6..."
...,...,...
2024-08-31 19:00:00,"[2486.277744, 2473.212192, 2460.2153, 2447.286...","[2525.89, 2539.23384, 2552.648173, 2566.133372..."
2024-08-31 20:00:00,"[2486.277744, 2473.212192, 2460.2153, 2447.286...","[2525.89, 2539.23384, 2552.648173, 2566.133372..."
2024-08-31 21:00:00,"[2499.41232, 2486.277744, 2473.212192, 2460.21...","[2539.23384, 2552.648173, 2566.133372, 2579.68..."


In [16]:
res_1h[window_hours:]

,price,quote,base,mtm,volume,mtm_eth,base_returns,quote_returns
time,,,,,,,,
2024-07-02 23:00:00,3416.34,37500.000000,10.976659,75000.000000,0.0,21.953318,0.000000,0.000000
2024-07-03 00:00:00,3419.79,78758.181244,-1.097666,75004.394034,0.0,21.932456,-0.000950,0.000059
2024-07-03 01:00:00,3389.99,59999.997157,4.419458,74981.915583,0.0,22.118624,0.007530,-0.000241
2024-07-03 02:00:00,3377.63,52499.997425,6.641262,74931.723194,0.0,22.184704,0.010540,-0.000910
2024-07-03 03:00:00,3355.62,45032.756392,8.872771,74806.404215,0.0,22.292871,0.015467,-0.002581
...,...,...,...,...,...,...,...,...
2024-08-31 19:00:00,2499.79,58612.904436,2.605630,65126.432254,0.0,26.052761,0.186735,-0.131648
2024-08-31 20:00:00,2506.80,58612.904436,2.605630,65144.697720,0.0,25.987194,0.183748,-0.131404
2024-08-31 21:00:00,2517.80,65177.145999,0.000000,65177.145999,0.0,25.886546,0.179163,-0.130971


In [17]:
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(""),
    specs=[[{"secondary_y": True}]],
)

for i, ob in enumerate(order_book_history_1h.iterrows()):
    bids = ob[1].bids
    asks = ob[1].asks

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(bids))],
            y=tuple(bids),
            mode="markers",
            name="Bids",
            showlegend=False,
            marker=dict(
                color="green",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(asks))],
            y=tuple(asks),
            mode="markers",
            name="Asks",
            showlegend=False,
            marker=dict(
                color="red",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(order_book_history_1h))],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
        marker=dict(color="blue"),
    )
)

fig.update_layout(
    height=1000,
    width=1600,
    title_text=f"Order book<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.show()

In [18]:
hv_1h = historical_vol.resample("1h").last()

In [19]:
bvol_copy = implied_vol.copy()
bvol_copy.index = [datetime.fromtimestamp(ts) for ts in implied_vol.index]
bvol_1h = bvol_copy.resample("1h").last()
bvol_1h[-len(res_1h) - 2 : -2]

,bvol
2024-08-01 02:00:00,57.2228
2024-08-01 03:00:00,57.1855
2024-08-01 04:00:00,56.8511
2024-08-01 05:00:00,56.4021
2024-08-01 06:00:00,56.6483
...,...
2024-08-31 19:00:00,66.2410
2024-08-31 20:00:00,66.2116
2024-08-31 21:00:00,66.1958
2024-08-31 22:00:00,66.2634


In [20]:
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(""),
    specs=[[{"secondary_y": True}]],
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=res_1h.price,
        mode="lines",
        name="Price",
        marker=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=bvol_1h.index[-len(res_1h) - 2 : -2],
        y=bvol_1h.bvol,
        mode="lines",
        name="IV",
        marker=dict(color="grey"),
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index,
        y=hv_1h[-len(res_1h) :],
        mode="lines",
        name="HV",
        marker=dict(color="purple"),
    ),
    secondary_y=True,
)

fig.update_layout(
    height=1000,
    width=1600,
)

fig.show()